[Make link](#Make-link)  
[To objects](#to-objects)  
[one to two file](#one-to-two-file)  
[copy rename files](#copy-rename-files)  
[](#)  
[](#)  
[](#)  
[](#)  
[](#)  


## makeLink
## Make link  

In [7]:
import re

def makeLink(s):
    s2 = s
    s2 = re.sub(r' ', '-', s)

    return f'[{s}](#{s2})  '

s = 'copy rename files'
s2 = makeLink(s)
print(s2)


[copy rename files](#copy-rename-files)  


# Time twice

# To objects

In [2]:
import re
import json
from deep_translator import GoogleTranslator as google_translator

translator = google_translator(source='auto', target='ru')  

class Lines:
    '''
    nextName, readLines, read, write, writeLines, mIntervals, , ,
    '''
    name = ''
    nameMask = ''
    lines = []
    intervalLines = [] 
    text = ""
    lines2 = [] # копия входных данных
    objLines = []
    enRuObjLines = []
    objText = ''
    enLines = []  
    ruLines = []
    enRuLines = []

    def __init__(self, name=""):

        self.name = ''
        self.nameMask = ''
        self.lines = []
        self.intervalLines = [] 
        self.text = ""
        self.lines2 = [] # копия входных данных
        self.objLines = []
        self.enRuObjLines = []
        self.objText = ''
        self.enLines = []  
        self.ruLines = []
        self.enRuLines = []


        self.name = name
        self.nameMask = self.nextName(self.name)

    def mEnRuObjLines(self, objLines, enRuLines, enRuObjLines):
        def find(a, l):
            r = ''
            l2 = list(iter(filter(lambda x: x[0] == a, l)))
            if len(l2) > 0:
                r = l2[0][1]
            return r


        for e in objLines:
            enRuObjLines.append(e)

        for e in enRuObjLines:
            li = []
            for e2 in e['en']:
                if len( e2 ) > 0:
                    li.append(find(e2, enRuLines))
                
            if len(li) == 0:
                e['ru'] = [""]
            else:
                e['ru'] = li
                

                pass
        pass

    def mTranslate(self, vStrings, vCouples):
        textEn = '\n'.join(vStrings)
        textRu = translator.translate(textEn)
        ruStrings = textRu.split('\n')
        k = 0
        for e in ruStrings:
            vCouples.append([vStrings[k], e])
            k += 1
        pass

    def mTranslateBig(self, vStrings, vCouples):
        max = 3000
        allParts = []
        sum = 0
        k = 0
        parts = []
        for e in vStrings:
            sum += len(e)
            parts.append(e)
            if sum > max or k == len(vStrings) - 1:
                allParts.append(parts)
                sum = 0
                parts = []
            k += 1

        for parts in allParts:
            self.mTranslate(parts, vCouples)


            pass



        pass




    def getEnLines(self, objLines, enLines):
        enLines1 = []
        for e in self.objLines:
            for e2 in e['en']:
                enLines1.append(e2)
        
        enLines2 = []
        begin = True
        for e in enLines1:
            if len(e) == 0 and begin:
                pass
            else:
                begin = False
                enLines2.append(e)

        enLines2.reverse()
        enLines3 = []
        begin = True
        for e in enLines2:
            if len(e) == 0 and begin:
                pass
            else:
                begin = False
                enLines3.append(e)

        enLines3.reverse()

        for e in enLines3:
            enLines.append(e)


        



    def nextName(self, name):
        name2 = ''
        patten = r'^\w\d\d'
        match = re.search(patten, name)
        name2 = match[0] if match else 'w000'
        return name2

    def readLines(self, name, items):
        with open(name, 'r', encoding='utf8') as file:
            for line in file:
                line2 = line.strip()
                len(line2) == 0 or items.append(line2)
       

    def read(self, name):
        self.lines = []
        self.readLines(name, self.lines)
        self.text = '\n'.join(self.lines)
        return self.text

    def write(self, name, text):
        with open(name, 'w', encoding='utf8') as file:
            file.write(text)

    def writeLines(self, name, items):
        text = '\n'.join(items)
        with open(name, 'w', encoding='utf8') as file:
            file.write(text)

    def mIntervals(self, l1, l2):
        # print(l1)
        '''
        [1,,2,,3] -> [1,2,,2,3,,3,3]
        '''
        l1.reverse()

        sPrev = ''
        patten = '^\d+\.\d+$'
        for e in l1:
            match = re.search(patten, e)
            if match:
                sPrev = e
                break

        k = 0
        for e in l1:
            match = re.search(patten, e)
            if match:
                l2.append(sPrev)
                sPrev = e
            l2.append(e)

        l1.reverse()
        l2.reverse()
        pass

    def mRu(self, text):
        ru = False
        ru = True if re.search('^[а-яА-Я]', text) else False
        return ru
        pass

    def mEn(self, text):
        en = False
        en = True if re.search('^[a-zA-Z]', text) else False
        return en
        pass

    def mTime(self, text):
        time = False
        patten = '^\d+\.\d+$'
        time = True if re.search(patten, text) else False
        return time




    def mObj(self, l1, l2):
        obj0 = {"time":[], "en": [], "ru": [] }
        l1.append("")

        k = 0
        obj = 0
        max = len(l1)
        en = []
        ru = []
        while k < max-2:
            notRun = True
            n = 0
            while k < max - 1 and self.mTime(l1[k]) and n < 2:
                if n == 0:
                    obj = obj0.copy()
                    obj["time"] = []
                    en = []
                    ru = []
                obj["time"].append(float(l1[k]))
                n += 1
                k += 1
                notRun = False

            while k < max - 1 and self.mEn(l1[k]):
                en.append(l1[k])
                obj["en"] = en
                k +=1
                notRun = False

            while k < max - 1 and self.mRu(l1[k]):
                ru.append(l1[k])
                obj["ru"] = ru
                k +=1
                notRun = False
            
            if len(obj['ru']) == 0:
                obj['ru'] = [""]
            if len(obj['en']) == 0:
                obj['en'] = [""]

            l2.append(obj)
            if notRun:
                k +=1

        pass

    def mObjText(self, objl):
        k = 0
        text = json.dumps([ob for ob in objl], indent=2, ensure_ascii=False)
        return text

    def srToLines(self, items, notes):
        for e in items:
            s = e
            match = re.search(r'(\d+):(\d+)', e)
            if match:
                # print(match[1], match[2])
                t = int(match[1])*60+int(match[2])
                s = f'{t}.00'
            notes.append(s)
        pass

    def srtToLines(self, items, notes):

        items.append('aaaaaaaaa')
        max = len(items)

        note = []
        # notes = []
        for i in range(max):
            e = items[i]

            pattern = r'(\d+)\:(\d+)\:(\d+)\,(\d+)'
            match = re.search(pattern, e)
            
            if match or i == max - 1:

                kk = 3
                if len(note) > kk:
                    notes.append(' '.join(note))
                    note = []
                elif len(note) <= kk and len(note) > 0:
                    t = notes.pop()
                    t = notes.pop()
                    notes.append(t + ' ' + ' '.join(note))
                pass


            if match:
                m = int(match[2])
                s = float(match[3] + '.' + match[4])
                time = m * 60 + s
                notes.append(str(time))
                continue

            match = re.search(r'^\d+$', e)
            if match:
                continue

            note += e.split(' ')
            note = list(filter(lambda e: len(e)>0, note))


        pass
       

lines = Lines('b02b.txt')
lines.lines = []

lines.readLines(lines.name, lines.lines)


notes = []
if(0):
    lines.srtToLines(lines.lines, notes)
else:
    lines.srToLines(lines.lines, notes)

lines.lines = notes


lines.mIntervals(lines.lines, lines.intervalLines)

lines.mObj(lines.intervalLines, lines.objLines)
lines.objText = lines.mObjText(lines.objLines)
lines.write(lines.nameMask + 'bb.txt', lines.objText)
lines.getEnLines(lines.objLines, lines.enLines)
lines.writeLines(lines.nameMask + 'c1.txt', lines.enLines)


lines.mTranslateBig(lines.enLines, lines.enRuLines)

lines.mEnRuObjLines(lines.objLines, lines.enRuLines, lines.enRuObjLines)
objText = lines.mObjText(lines.enRuObjLines)
lines.write(lines.nameMask + 'd.txt', objText)

print(len(lines.objText), objText[:200])


295 [
  {
    "time": [
      0.0,
      63.0
    ],
    "en": [
      "welcome to this free Cisco CCNA packet"
    ],
    "ru": [
      "добро пожаловать в этот бесплатный пакет Cisco CCNA"
    ]
  },
  


## one to two file


In [8]:
def oneToTwo(name1, name2):
    lines = Lines(name1)
    lines.lines = []

    lines.readLines(lines.name, lines.lines)

    notes = []
    if(0):
        lines.srtToLines(lines.lines, notes)
    else:
        lines.srToLines(lines.lines, notes)

    lines.lines = notes
    # print(lines.lines)

    lines.mIntervals(lines.lines, lines.intervalLines)

    lines.mObj(lines.intervalLines, lines.objLines)
    lines.objText = lines.mObjText(lines.objLines)
    lines.write(lines.nameMask + 'bb.txt', lines.objText)
    lines.getEnLines(lines.objLines, lines.enLines)
    lines.writeLines(lines.nameMask + 'c1.txt', lines.enLines)


    lines.mTranslateBig(lines.enLines, lines.enRuLines)

    lines.mEnRuObjLines(lines.objLines, lines.enRuLines, lines.enRuObjLines)
    objText = lines.mObjText(lines.enRuObjLines)
    objText = 'let objs = ' + objText
    lines.write(name2, objText)



    pass

oneToTwo('b03b.txt', 'b03d.txt')
oneToTwo('b03b.txt', 'b03d2.txt')


https://habr.com/ru/post/349860/

## copy rename files

In [3]:
import os
import re
import shutil

class Wood:
    extension = ''
    path = 'aa'
    inFiles = []
    filess = []
    log = ''

    def __init__(self, extension = 'txt'):
        self.extension = extension

    def setPath(self, path = ''):
        self.path = self.path if len(path) == 0 else path


    def getInFiles(self):
        files = os.listdir(self.path)
        print(files)
        files = list(filter(lambda e: re.search(r'\.sr$', e), files))
        timeFiles = []
        for e in files:
            time = os.path.getmtime(self.path+'/'+e)
            timeFiles.append([e, time])
            timeFiles
            # print(os.path.getmtime(e))

        sortFiles = sorted(timeFiles, key = lambda e: e[1])
        self.inFiles = list(map(lambda e: e[0], sortFiles))
        pass

    def mNames(self, firstName, vFiles = []):
        vFiles = self.inFiles if len(vFiles) == 0 else vFiles
        max = len(vFiles)
        self.filess = list(map(lambda e: e, vFiles))

        pattern = r'^(([A-Za-z]+)(\d+))'
        match = re.search(pattern, firstName)
        mask = match[2]
        num = match[3]
        # print(num)
        num = int(num)

        nNames = []
        for i  in range(num, num + max):
            sNum = f'{i:02d}'
            nNames.append(mask + sNum)
            pass

        self.filess = list(map(lambda a, b: [a, b], nNames, vFiles))

        # print(nNames)
        # print(mask, num, max, firstName)
        pass

    def copy(self, filess = []):
        filess = self.filess if len(filess) == 0 else filess
        for ee in filess:
            shutil.copy2('aa/' + ee[1], 'bb/' + ee[0] + '.txt')
            pass
        pass

    def copyTranslate(self, filess = []):
        filess = self.filess if len(filess) == 0 else filess
        for ee in filess:
            oneToTwo('aa/' + ee[1], 'bb/' + ee[0] + 'items.js')
            # shutil.copy2('aa/' + ee[1], 'bb/' + ee[0] + '.txt')
            pass
        pass

    def write(self, name, text=''):
        text0 = ''
        for e in self.filess:
            text0 += e[0]
            text0 += '\n'
            text0 += e[1]
            text0 += '\n'
            pass
        text = text0  if len(text) == 0 else text
        with open(name, 'a', encoding='utf8') as file:
            file.write(text)

        self.log += text

        # print(text)
        pass

w = Wood('sr')
w.getInFiles()
w.mNames('k01')
# w.copy()
w.copyTranslate()
w.write('p05srt.txt')
print(w.log[:400])
print(w.filess)


['k01a.sr', 'k02a.sr', 'k03a.sr', 'k04a.sr', 'k05a.sr', 'k06a.sr', 'k07a.sr', 'k08a.sr', 'k09a.sr', 'k10a.sr', 'k11a.sr', 'k12a.sr', 'k13a.sr', 'k14a.sr', 'k15a.sr', 'k16a.sr', 'k17a.sr', 'k18a.sr', 'k19a.sr', 'k20a.sr', 'k21a.sr', 'k22a.sr', 'k23a.sr', 'k24a.sr', 'k25a.sr', 'k26a.sr', 'k27a.sr', 'k28a.sr', 'k29a.sr', 'k30a.sr', 'k31a.sr', 'k32a.sr', 'k33a.sr', 'k34a.sr', 'k35a.sr', 'k36a.sr', 'k37a.sr', 'k38a.sr', 'k39a.sr', 'k40a.sr', 'k41a.sr', 'k42a.sr', 'k43a.sr', 'k44a.sr', 'k45a.sr', 'k46a.sr', 'k47a.sr', 'k48a.sr', 'k49a.sr', 'k50a.sr', 'k51a.sr', 'k52a.sr', 'k53a.sr', 'k54a.sr', 'k55a.sr', 'k56a.sr', 'k57a.sr', 'k58a.sr', 'k59a.sr', 'k60a.sr', 'k61a.sr', 'k62a.sr', 'k63a.sr', 'k64a.sr', 'k65a.sr', 'k66a.sr', 'k67a.sr', 'k68a.sr', 'k69a.sr', 'k70a.sr', 'k71a.sr', 'k72a.sr', 'k73a.sr', 'k74a.sr', 'k75a.sr', 'k76a.sr']


NameError: name 'oneToTwo' is not defined

## translate modifyfiles

In [5]:
import os
import re
import modifyfiles as mf0

def modifyLine(text):
    text2 = text

    return text2

def modifyText(text):
    '''

    '''
    text2 = text 
    lines = Lines()
    lines.lines = text.split('\n')

    # lines.readLines(lines.name, lines.lines)

    notes = []
    if(0):
        lines.srtToLines(lines.lines, notes)
    else:
        lines.srToLines(lines.lines, notes)

    lines.lines = notes
    # print(lines.lines)

    lines.mIntervals(lines.lines, lines.intervalLines)

    lines.mObj(lines.intervalLines, lines.objLines)
    lines.objText = lines.mObjText(lines.objLines)
    lines.write(lines.nameMask + 'bb.txt', lines.objText)
    lines.getEnLines(lines.objLines, lines.enLines)
    lines.writeLines(lines.nameMask + 'c1.txt', lines.enLines)


    lines.mTranslateBig(lines.enLines, lines.enRuLines)

    lines.mEnRuObjLines(lines.objLines, lines.enRuLines, lines.enRuObjLines)
    objText = lines.mObjText(lines.enRuObjLines)
    objText = 'let objs = ' + objText
    # lines.write(name2, objText)

    return objText    

# mf = ModifyFiles()
mf = mf0.ModifyFiles()
mf.getFrPath('oo1').getToPath('oo2')
mf.getIndexName('k68a.txt')
inFiles = []
mf.getInFiles(inFiles)
mf.setPostfixNext('items.js')
outFiles = []
mf.getOutFiles(outFiles)
mf.setMax()

mf.setModifyLine()
mf.setModify(modifyText)

for i in range(mf.max):
    mf.modify(inFiles[i], outFiles[i])

print(inFiles)
print(outFiles)


['k68a.txt']
['k68items.js']


http://pythonicway.com/python-functinal-programming#filter

